# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
# !pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai
# !pip install -qU ragas
# !pip install -qU faiss_cpu pymupdf pandas

We'll also get the "star of the show" today, which is Ragas!

As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
import openai
# from getpass import getpass

# openai.api_key = getpass("Please provide your OpenAI Key: ")
# os.environ["OPENAI_API_KEY"] = openai.api_key\

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [4]:
# !git clone https://github.com/AI-Maker-Space/DataRepository

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

Answer:

-They use a compressed representation of the vectors rather than the original vectors. 

-The ability to use GPUs. 


> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Answer:

The first runnable chain is created to retrieve the top k chunks most similar to the prompt. That context is then passed to the llm to obtain a response.



Let's test it out!

In [18]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [19]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, accounting, and demand for a jury trial.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [ ]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

Answer: It can create broader / not as specific questions as each chunk is much smaller in the previous split of our documents .

In [21]:
len(documents)

159

In [22]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(eval_documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

/home/amanda.vanderwal/venvs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                   
Generating: 100%|██████████| 10/10 [02:19<00:00, 13.91s/it]


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

Answer:

The distrubution of each type of questions that are created in the test set. So, 50% of the created test samples will be simple, 25% reasoning, and 25% multi-context .


> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Let's look at the output and see what we can learn about it!

In [23]:
testset.test_data[0]

DataRow(question='What is the significance of the "Transformers" algorithm in the research conducted by OpenAI, Inc.?', contexts=['1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 7 – \nCOMPLAINT \n \nMusk continued to make contributions to OpenAI, Inc. from its founding through September 14, \n2020. \n27. \nOpenAI’s initial research was performed in the open, providing free and public \naccess to designs, models, and code. When OpenAI, Inc. researchers discovered that an algorithm \ncalled “Transformers,” initially invented by Google, could perform many natural language tasks \nwithout any explicit training, entire communities sprung up to enhance and extend the models \nreleased by OpenAI, Inc. These communities spread to open-source, grass-roots efforts and'], ground_truth='The significance of the "Transformers" algorithm in the research conducted by OpenAI, Inc. is that it can perform many nat

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [24]:
test_df = testset.to_pandas()

In [25]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,"What is the significance of the ""Transformers""...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"The significance of the ""Transformers"" algorit...",simple,True
1,What strategy video game did OpenAI compete in?,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,True
2,What concerns do US officials have about OpenA...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about OpenAI's inve...,reasoning,True
3,How did OpenAI use reinforcement learning in t...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,reasoning,True
4,What concern did Mr. Musk express about AI dur...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Musk expressed concern about the dangers o...,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,"OpenAI used Dota 2, a strategy video game, to ...",multi_context,True
6,What principles in the OpenAI Founding Agreeme...,[profit developing AGI for the benefit of huma...,The OpenAI Founding Agreement prioritizes AGI ...,multi_context,True
7,"""What role did the ""175 billion parameters"" pl...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,multi_context,True
8,What makes AGI in the wrong hands dangerous an...,[18. \nMr. Musk has long recognized that AGI p...,The advancement of AI in the wrong hands is da...,multi_context,True
9,What's the name of Mr. Musk's new lab that pri...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,reasoning,True


In [26]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [27]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [28]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [29]:
response_dataset[0]

{'question': 'What is the significance of the "Transformers" algorithm in the research conducted by OpenAI, Inc.?',
 'answer': 'The "Transformers" algorithm discovered by Google was found to perform many natural language tasks without explicit training, leading to the development of new models and communities that enhanced and extended the models released by OpenAI, Inc.',
 'contexts': ['1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 7 – \nCOMPLAINT \n \nMusk continued to make contributions to OpenAI, Inc. from its founding through September 14, \n2020. \n27. \nOpenAI’s initial research was performed in the open, providing free and public \naccess to designs, models, and code. When OpenAI, Inc. researchers discovered that an algorithm \ncalled “Transformers,” initially invented by Google, could perform many natural language tasks \nwithout any explicit training, entire communities sprung up to e

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [30]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [31]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 50/50 [00:28<00:00,  1.77it/s]


In [32]:
results

{'faithfulness': 0.8000, 'answer_relevancy': 0.9201, 'context_recall': 0.8000, 'context_precision': 0.7472, 'answer_correctness': 0.7359}

In [33]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the significance of the ""Transformers""...","The ""Transformers"" algorithm discovered by Goo...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"The significance of the ""Transformers"" algorit...",1.0,0.832832,1.0,0.333333,0.613330
1,What strategy video game did OpenAI compete in?,Dota 2,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",1.0,0.953850,1.0,1.000000,0.716374
2,What concerns do US officials have about OpenA...,US officials are concerned about OpenAI's inve...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about OpenAI's inve...,0.0,0.967384,1.0,1.000000,1.000000
3,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,1.0,0.903244,1.0,1.000000,0.745112
4,What concern did Mr. Musk express about AI dur...,Mr. Musk expressed concerns about the dangers ...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Musk expressed concern about the dangers o...,1.0,0.934226,1.0,1.000000,0.999420
5,"""What game did OpenAI use reinforcement learni...",Dota 2,[a superhuman level of play in the games of ch...,"OpenAI used Dota 2, a strategy video game, to ...",0.0,0.897363,0.0,0.500000,0.961786
6,What principles in the OpenAI Founding Agreeme...,The principles in the OpenAI Founding Agreemen...,[profit developing AGI for the benefit of huma...,The OpenAI Founding Agreement prioritizes AGI ...,1.0,0.925227,1.0,1.000000,0.740986
7,"""What role did the ""175 billion parameters"" pl...","The ""175 billion parameters"" played a signific...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,1.0,0.965910,1.0,0.833333,0.911239
8,What makes AGI in the wrong hands dangerous an...,AGI in the wrong hands is dangerous and an exi...,[18. \nMr. Musk has long recognized that AGI p...,The advancement of AI in the wrong hands is da...,1.0,0.979789,1.0,0.805556,0.479033
9,What's the name of Mr. Musk's new lab that pri...,OpenAI,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,1.0,0.841301,0.0,0.000000,0.191701


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [34]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [35]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [36]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [37]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [38]:
print(response["answer"])

The plaintiff is Elon Musk.


In [39]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [40]:
print(response["answer"])

The complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices, and seeking accounting, restitution, disgorgement of funds, and injunctive relief against all Defendants. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [41]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [42]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [43]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


In [44]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the significance of the ""Transformers""...","The ""Transformers"" algorithm, initially invent...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,"The significance of the ""Transformers"" algorit...",0.75,0.903925,1.0,0.500000,0.535857
1,What strategy video game did OpenAI compete in?,"OpenAI competed in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",1.00,1.000000,1.0,1.000000,0.741034
2,What concerns do US officials have about OpenA...,US officials are concerned about OpenAI's ties...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about OpenAI's inve...,NaN,0.952490,1.0,0.950000,0.746249
3,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,1.00,0.895773,1.0,1.000000,0.615428
4,What concern did Mr. Musk express about AI dur...,"During his conversation with Mr. Page, Mr. Mus...","[Page, then-CEO of Google’s parent company Alp...",Mr. Musk expressed concern about the dangers o...,1.00,0.965845,1.0,0.950000,0.745183
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to surpass ...,[a superhuman level of play in the games of ch...,"OpenAI used Dota 2, a strategy video game, to ...",NaN,0.946179,0.0,0.500000,0.615086
6,What principles in the OpenAI Founding Agreeme...,The principles in the OpenAI Founding Agreemen...,[profit developing AGI for the benefit of huma...,The OpenAI Founding Agreement prioritizes AGI ...,1.00,0.921687,1.0,1.000000,0.537153
7,"""What role did the ""175 billion parameters"" pl...","The ""175 billion parameters"" in OpenAI's GPT-3...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The publication of the research paper describi...,1.00,0.966136,1.0,0.833333,0.871089
8,What makes AGI in the wrong hands dangerous an...,AGI in the wrong hands is considered dangerous...,[18. \nMr. Musk has long recognized that AGI p...,The advancement of AI in the wrong hands is da...,1.00,0.957879,1.0,0.666667,0.895956
9,What's the name of Mr. Musk's new lab that pri...,The name of Mr. Musk's new lab that prioritize...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,nan,1.00,0.999155,1.0,0.000000,0.187053


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [45]:
results

{'faithfulness': 0.8000, 'answer_relevancy': 0.9201, 'context_recall': 0.8000, 'context_precision': 0.7472, 'answer_correctness': 0.7359}

And see how our advanced retrieval modified our chain!

In [46]:
advanced_retrieval_results

{'faithfulness': 0.9688, 'answer_relevancy': 0.9509, 'context_recall': 0.9000, 'context_precision': 0.7400, 'answer_correctness': 0.6490}

In [47]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.800000,0.968750,0.168750
1,answer_relevancy,0.920113,0.950907,0.030794
2,context_recall,0.800000,0.900000,0.100000
3,context_precision,0.747222,0.740000,-0.007222
4,answer_correctness,0.735898,0.649009,-0.086889


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

### Loading in the smaller OpenAI embedding models

In [48]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

### Create the FAISS vector store

In [49]:
vector_store = FAISS.from_documents(documents, new_embeddings)

### Create retriever

In [50]:
new_retriever = vector_store.as_retriever()

### Create an advanced retriever retriever that generates multiple queries for each input

In [51]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

### Create a retrievel chain 

In [52]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [53]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [54]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [55]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 50/50 [01:08<00:00,  1.38s/it]


In [56]:
new_advanced_retrieval_results

{'faithfulness': 0.8350, 'answer_relevancy': 0.9478, 'context_recall': 0.9000, 'context_precision': 0.7450, 'answer_correctness': 0.6618}

In [57]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.800000,0.968750,0.835000,-0.133750,0.035000
1,answer_relevancy,0.920113,0.950907,0.947799,-0.003108,0.027687
2,context_recall,0.800000,0.900000,0.900000,0.000000,0.100000
3,context_precision,0.747222,0.740000,0.745000,0.005000,-0.002222
4,answer_correctness,0.735898,0.649009,0.661758,0.012749,-0.074140


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

Answer: 

No, it's comparable and slightly worse in the fathfullness. 

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [62]:
tf = test_df['evolution_type'] == 'multi_context'
multi_context = test_df[tf].copy()

In [86]:
new_response_df = new_response_dataset_advanced_retrieval.to_pandas()

In [90]:
for question in multi_context['question']:
    print(question)

    tf = results_df['question'] == question
    print('Baseline')
    print(results_df[tf]['answer'].values[0])    

    tf = advanced_retrieval_results_df['question'] == question
    print('ADA Results')
    print(advanced_retrieval_results_df[tf]['answer'].values[0])

    tf = new_response_df['question'] == question
    print('Text Embedding 3 Results')
    print(new_response_df[tf]['answer'].values[0])
    print('\n\n')

What concern did Mr. Musk express about AI during his conversation with Mr. Page, and how did Mr. Page respond?
Baseline
Mr. Musk expressed concerns about the dangers of AI potentially replacing humans and making our species irrelevant or extinct. Mr. Page responded by saying it would merely be the next stage of evolution and accused Mr. Musk of being a "specist."
ADA Results
During his conversation with Mr. Page, Mr. Musk expressed concern about the dangers of AI, warning that artificial intelligence systems might replace humans, making our species irrelevant or even extinct if safeguards were not put in place. In response, Mr. Page dismissed these concerns by stating that it would merely be the next stage of evolution and accused Mr. Musk of being a "specist."
Text Embedding 3 Results
During his conversation with Mr. Page, Mr. Musk expressed concerns about the dangers of AI, warning that artificial intelligence systems might replace humans, making our species irrelevant or even extin